In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

In [55]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# PreProcessing

In [17]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [19]:
df.drop(['PassengerId','Name', 'Ticket', 'Cabin', 'Fare'], axis = 1, inplace = True)

In [20]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [21]:
def handle_nulls(df):
    for i in df.columns:
        if df[i].dtype.name == 'object':
            df[i].fillna(df[i].mode()[0], inplace = True)
        elif df[i].dtype.name == 'int64' or df[i].dtype.name == 'float64':
            df[i].fillna(df[i].mean(), inplace = True)

In [22]:
handle_nulls(df)

In [23]:
df['relatives'] = df['SibSp'] + df['Parch'] 

In [24]:
df.drop(['SibSp', 'Parch'],axis = 1, inplace = True)

In [26]:
df['Label'] = pd.cut(x=df['Age'], bins=[0, 3, 17, 30, 50, 99], labels=['Baby', 'Child', 'Young', 'Adult', 'Elderly'])

In [27]:
df.drop(['Age'], axis = 1, inplace = True)

In [28]:
df_encoded = pd.get_dummies(df, columns = ['Pclass', 'Sex', 'Embarked', 'Label'])

In [30]:
df_encoded.drop(['Sex_female'], axis = 1, inplace = True)

In [31]:
df_encoded

,Survived,relatives,Pclass_1,Pclass_2,Pclass_3,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Label_Baby,Label_Child,Label_Young,Label_Adult,Label_Elderly
0,0,1,0,0,1,1,0,0,1,0,0,1,0,0
1,1,1,1,0,0,0,1,0,0,0,0,0,1,0
2,1,0,0,0,1,0,0,0,1,0,0,1,0,0
3,1,1,1,0,0,0,0,0,1,0,0,0,1,0
4,0,0,0,0,1,1,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,0,0,1,0,1,0,0,1,0,0,1,0,0
887,1,0,1,0,0,0,0,0,1,0,0,1,0,0
888,0,3,0,0,1,0,0,0,1,0,0,1,0,0
889,1,0,1,0,0,1,1,0,0,0,0,1,0,0


In [32]:
X = df_encoded.loc[:, df_encoded.columns != 'Survived']
y = df_encoded.loc[:, 'Survived']

In [33]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [35]:
x_train.shape

(596, 13)

# XGBoost

In [36]:
from xgboost import XGBClassifier

In [37]:
bst = XGBClassifier()

In [38]:
bst.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [50]:
y_pred = bst.predict(x_test)

In [51]:
y_pred

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0])

In [54]:
accuracy_score(y_test, y_pred)

0.7830508474576271

In [128]:
# pca = PCA(n_components = 8)

In [129]:
# x_train_norm_pca = pca.fit_transform(x_train_norm)
# x_test_norm_pca = pca.transform(x_test_norm)

In [130]:
# lgr_model = LogisticRegression()
# lgr_model.fit(x_train_pca, y_train)
# lgr_model.score(x_test_pca, y_test)

In [56]:
model = Sequential([
    Dense(100, activation = "relu"),
    Dense(50, activation = "relu"),
    Dense(1, activation = "sigmoid")
], name = 'my_model')

In [57]:
model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
             optimizer=tf.keras.optimizers.Adam(0.001)
             )

In [59]:
model.fit(x_train, y_train, epochs = 50)

Epoch 1/50
19/19 [==============================] - 4s 5ms/step - loss: 0.6514
Epoch 2/50
19/19 [==============================] - 0s 4ms/step - loss: 0.5794
Epoch 3/50
19/19 [==============================] - 0s 4ms/step - loss: 0.5385
Epoch 4/50
19/19 [==============================] - 0s 4ms/step - loss: 0.5032
Epoch 5/50
19/19 [==============================] - 0s 4ms/step - loss: 0.4738
Epoch 6/50
19/19 [==============================] - 0s 4ms/step - loss: 0.4553
Epoch 7/50
19/19 [==============================] - 0s 5ms/step - loss: 0.4385
Epoch 8/50
19/19 [==============================] - 0s 4ms/step - loss: 0.4280
Epoch 9/50
19/19 [==============================] - 0s 4ms/step - loss: 0.4234
Epoch 10/50
19/19 [==============================] - 0s 4ms/step - loss: 0.4214
Epoch 11/50
19/19 [==============================] - 0s 4ms/step - loss: 0.4125
Epoch 12/50
19/19 [==============================] - 0s 4ms/step - loss: 0.4091
Epoch 13/50
19/19 [==============================

In [52]:
y_train_pred = model.predict(x_test_norm)
for i in range(len(y_train_pred)):
    y_train_pred[i] = 1 if y_train_pred[i] > 0.5 else 0

10/10 [==============================] - 0s 2ms/step


In [60]:
model.evaluate(x_test, y_test)

10/10 [==============================] - 0s 5ms/step - loss: 0.4503


0.45033764839172363

# Test

In [7]:
df_test = pd.read_csv('test.csv')
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [9]:
pass_id = df_test['PassengerId']

In [10]:
df_test.drop(['PassengerId','Name', 'Ticket', 'Cabin'], axis = 1, inplace = True)

In [11]:
df_test.isnull().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

In [17]:
handle_nulls(df_test)

In [18]:
df_test_encoded = pd.get_dummies(df_test, columns = ['Sex', 'Embarked']

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,34.50000,0,0,7.8292,1,0,1,0
1,3,47.00000,1,0,7.0000,0,0,0,1
2,2,62.00000,0,0,9.6875,1,0,1,0
3,3,27.00000,0,0,8.6625,1,0,0,1
4,3,22.00000,1,1,12.2875,0,0,0,1
...,...,...,...,...,...,...,...,...,...
413,3,30.27259,0,0,8.0500,1,0,0,1
414,1,39.00000,0,0,108.9000,0,1,0,0
415,3,38.50000,0,0,7.2500,1,0,0,1
416,3,30.27259,0,0,8.0500,1,0,0,1


In [19]:
df_test_encoded['relatives'] = df_test_encoded['SibSp'] + df_test_encoded['Parch']
df_test_encoded.drop(['SibSp', 'Parch'], axis = 1, inplace = True)

In [24]:
test_x = df_test_encoded.loc[:, df_test_encoded.columns].to_numpy()

In [25]:
test_x_norm = scaler.transform(test_x)

In [111]:
test_x_norm_pca = pca.transform(test_x_norm)

In [150]:
# y_pred = lgr_model.predict(test_x_norm_pca)

In [94]:
y_pred = model.predict(test_x_norm_pca)

14/14 [==============================] - 0s 2ms/step


In [95]:
for i in range(len(y_pred)):
    y_pred[i] = 1 if y_pred[i] > 0.5 else 0

In [102]:
y_pred = y_pred.astype(np.int64)

In [103]:
pass_id.values.resize(418,)

In [104]:
y_pred.resize(418,)

In [105]:
y_pred

array([0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [106]:
submission_df = pd.DataFrame({'PassengerId' : pass_id.values, "Survived" : y_pred})

In [107]:
submission_df.to_csv("submission.csv", index = False)